In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests

import spacy
nlp = spacy.load('en_core_web_lg')

In [2]:
import string
import re

In [143]:
###MONSTER DATASET CLEANING
monster = pd.read_csv('monster.csv')
monster['skills'] = monster['skills'].to_string()
cleaning = pd.DataFrame(columns=['lower','num'])

cleaning['liststring'] = [','.join(map(str, l)) for l in monster['skills']]
cleaning['lower'] = [i.lower() for i in monster['skills']]
cleaning['num'] = [re.sub(r'\w*\d\w*', '', i).strip() for i in cleaning['lower']]
monster['skills'] = [s.translate(str.maketrans('','',string.punctuation)) for s in cleaning['num']]


In [153]:
###JOBSTREET DATASET
jobstreet = pd.read_csv('jobstreet.csv')
jobstreet = jobstreet.rename(columns={'Skills':'skills'})

In [168]:
initial_df = pd.concat([monster, jobstreet], axis=0)
initial_df.head(10)
#initial_df.to_csv('combined.csv')

In [3]:
initial_df = pd.read_csv('combined.csv')

In [4]:
skills_doc = []
for x in initial_df['skills'].iloc[0:]:
    skills_doc.append(x)

skills_doc = ' '.join([str(elem) for elem in skills_doc])

In [6]:
from spacy.lang.en.stop_words import STOP_WORDS
stop = ["data",'business','candidate', 'experience', 'work', 'analysis','knowledge','tools','working','years','science','must','models','required','script','analytics','support','design','development','management','skills','degree','andor','reports','including','communication',\
       'related','information','requirements','team','performance','learning','ability','process','processes','technology','analyze','sales','risk','related','new','client','project','operations','written','quality','nan','technical','time','computer','field','based','insights',\
       'engineering','complex','strong','high','statistical','provide','financial','understanding','leadership','office','customer','employees','communicate','etc','clients','identify','develop','operational','possess','multiple','able','results','internal','stakeholders','ensure',\
       'systems','groups','bachelorscollege','company', 'partners','approach','model','end','ie','projects','intermediateexpert','relevant','key','benefits','global','familiarity','software','issues','advanced','resource','processing','maintain','datasets','meet','wellness','database',\
       'specific','test','sss','creating','compensation','big','trends','solutions','power','teams','large','quantitative','job','regular','assigned','use','sources','variable','tool','actionable','appropriate','metrics','output','finance','problem','home','system','good','understand',\
       'predictive','execution','campaign', 'planning','organization','bir','attention','documents','analyzing','improve','order','level','source','positions','problems','verbal','graduate','different','solving','requests','available','training','findings','implementation','account',\
       'platforms','help','improvement','implement','generate','ms','equivalentat','delivery','plans','existing','center','opportunities','external','accuracy','effectively','report','technologies','collaboration','accurate','excellent','closely','philhealth','pagibig','id','experienced'\
       'certificate','equivalentfulltime','computertelecommunication','scienceinformation','nurse','handson','payroll','implementations','understoodensure','prepare','takes','commission','ortigas','list','user','validate','profile','share','open','positionpreferably','senior','tin','certificate']

for word in stop:
    STOP_WORDS.add(word)

In [9]:
skills_doc = nlp(skills_doc)

ValueError: [E088] Text of length 52216713 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [8]:
clean_doc=[]
    
for token in skills_doc:
    if token.is_stop==False:
        clean_doc.append(token.text)

doc = " ".join(clean_doc)

AttributeError: 'str' object has no attribute 'is_stop'

In [157]:
nlp.max_length = 55000000

doc1 = nlp(skills_doc, disable = ['ner', 'parser'])

MemoryError: Unable to allocate 9.00 GiB for an array with shape (8390192, 288) and data type float32

In [103]:
from spacy.lang.en.stop_words import STOP_WORDS
stop = ["data",'business','candidate', 'experience', 'work', 'analysis','knowledge','tools','working','years','science','must','models','required','script','analytics','support','design','development','management','skills','degree','andor','reports','including','communication',\
       'related','information','requirements','team','performance','learning','ability','process','processes','technology','analyze','sales','risk','related','new','client','project','operations','written','quality','nan','technical','time','computer','field','based','insights',\
       'engineering','complex','strong','high','statistical','provide','financial','understanding','leadership','office','customer','employees','communicate','etc','clients','identify','develop','operational','possess','multiple','able','results','internal','stakeholders','ensure',\
       'systems','groups','bachelorscollege','company', 'partners','approach','model','end','ie','projects','intermediateexpert','relevant','key','benefits','global','familiarity','software','issues','advanced','resource','processing','maintain','datasets','meet','wellness','database',\
       'specific','test','sss','creating','compensation','big','trends','solutions','power','teams','large','quantitative','job','regular','assigned','use','sources','variable','tool','actionable','appropriate','metrics','output','finance','problem','home','system','good','understand',\
       'predictive','execution','campaign', 'planning','organization','bir','attention','documents','analyzing','improve','order','level','source','positions','problems','verbal','graduate','different','solving','requests','available','training','findings','implementation','account',\
       'platforms','help','improvement','implement','generate','ms','equivalentat','delivery','plans','existing','center','opportunities','external','accuracy','effectively','report','technologies','collaboration','accurate','excellent','closely']

for word in stop:
    nlp.vocab[word].is_stop = True

In [104]:
clean_doc=[]
    
for token in doc1:
    if token.is_stop==False:
        clean_doc.append(token.text)

doc = " ".join(clean_doc)

In [105]:
doc = str(doc)
docA = doc.split(" ")

In [11]:
wordSet = set(docA)
wordDictA = dict.fromkeys(wordSet,0)

In [12]:
for word in docA:
    wordDictA[word] += 1

In [13]:
def computeTF(wordDict,bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bowCount)
    return tfDict

In [14]:
tfBowA = computeTF(wordDictA,docA)

In [15]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word,val in doc.items():
            if val > 0:
                idfDict[word] += 1
    for word,val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [16]:
idfs = computeIDF([wordDictA])

In [17]:
def computeTFIDF(tfBow,idfs):
    tfidf = {}
    for word,val in tfBow.items():
        tfidf[word]= val*idfs[word]
    return tfidf

In [18]:
tfidfBowA = computeTFIDF(tfBowA,idfs)

In [19]:
df = pd.DataFrame([tfidfBowA])
df = pd.melt(df)
df.head(20)

C:\Users\hbalcera\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3427: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  exec(code_obj, self.user_global_ns, self.user_ns)


,variable,value
0,,0.0
1,leave,0.0
2,updating,0.0
3,setscapable,0.0
4,unitsdocument,0.0
5,backgroundbasic,0.0
6,player,0.0
7,editorial,0.0
8,dependable,0.0
9,databasescreate,0.0


In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [106]:
tfidf = TfidfVectorizer()
response = tfidf.fit_transform([doc])

In [107]:
feature_names = tfidf.get_feature_names()
data = []
for col in response.nonzero()[1]:
    data.append([feature_names[col], response[0,col]])
    
df = pd.DataFrame(data, columns=['features', 'values'])

In [110]:

df.sort_values('values',ascending=False).head(20)

#df.to_csv('example jobs.csv')

,features,values
4627,sql,0.214373
4404,analytical,0.181917
4655,modeling,0.172644
4241,excel,0.162007
4633,machine,0.156279
4389,reporting,0.152188
4622,python,0.151916
4560,statistics,0.146461
4616,visualization,0.132551
4614,tableau,0.129006


In [167]:
skills_df = df.sort_values('values',ascending=False)
skills_df.to_csv('skillsdf.csv')